In [1]:
pip install experta

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
!sed -i 's/collections.Mapping/collections.abc.Mapping/' /usr/local/lib/python3.10/dist-packages/frozendict/__init__.py

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
#name / عمر شعبان يوسف حرب 
#ID /120222897
from experta import *
import time
class Environment(Fact):
    """Represents the environment a trainee interacts with."""
    pass

class Job(Fact):
    """Represents the job type for the trainee."""
    pass

class Feedback(Fact):
    """Indicates whether feedback is required."""
    pass

class StimulusSituation(Fact):
    """Represents inferred stimulus situation."""
    pass

class StimulusResponse(Fact):
    """Represents inferred stimulus response."""
    pass

class CandidateMedium(Fact):
    """Intermediate recommendation before conflict resolution."""
    pass

class Medium(Fact):
    """Final selected medium."""
    pass

class MediaAdvisor(KnowledgeEngine):
    def __init__(self):
        super().__init__()
        print("Initializing the Media Advisor Expert System...\n")
# Rule 1
    @Rule(Environment(environment='papers') |
          Environment(environment='manuals') |
          Environment(environment='documents') |
          Environment(environment='textbooks'))
    def Rule_1_verbal(self):
        print("Rule 1 fired: Environment -> stimulus situation = verbal.")
        self.declare(StimulusSituation(value="verbal",ts=time.time()))

# Rule 2
    @Rule(Environment(environment='pictures') |
          Environment(environment='illustrations') |
          Environment(environment='photographs') |
          Environment(environment='diagrams'))
    def Rule_2_visual(self):
        print("Rule 2 fired: Environment -> stimulus situation = visual.")
        self.declare(StimulusSituation(value="visual",ts=time.time()))
# Rule 3
    @Rule(Environment(environment='machines') |
          Environment(environment='buildings') |
          Environment(environment='tools'))
    def Rule_3_physical_object(self):
        print("Rule 3 fired: Environment -> stimulus situation = physical object.")
        self.declare(StimulusSituation(value="physical object",ts=time.time()))
# Rule 4
    @Rule(Environment(environment='numbers') |
          Environment(environment='formulas') |
          Environment(environment='computer programs'))
    def Rule_4_symbolic(self):
        print("Rule 4 fired: Environment -> stimulus situation = symbolic.")
        self.declare(StimulusSituation(value="sumbolic",ts=time.time()))
# Rule 5
    @Rule(Job(job='lecturing') |
          Job(job='advising') |
          Job(job='counselling'))
    def Rule_5_oral_response(self):
        print("Rule 5 fired: job -> stimulus response = oral.")
        self.declare(StimulusResponse(value="oral",ts=time.time()))
# Rule 6
    @Rule(Job(job='building') |
          Job(job='repairing') |
          Job(job='troubleshooting'))
    def Rule_6_hands_on(self):
        print("Rule 6 fired: job -> stimulus response = hands-on.")
        self.declare(StimulusResponse(value="hands-on",ts=time.time()))
# Rule 7
    @Rule(Job(job='writing') |
          Job(job='typing') |
          Job(job='drawing'))
    def Rule_7_documented_response(self):
        print("Rule 7 fired: job -> stimulus response = documented.")
        self.declare(StimulusResponse(value="documented",ts=time.time()))
# Rule 8
    @Rule(Job(job='evaluating') |
          Job(job='reasoning') |
          Job(job='investigating'))
    def Rule_8_analytical_response(self):
        print("Rule 8 fired: job -> stimulus response = analytical.")
        self.declare(StimulusResponse(value="analytical",ts=time.time()))
# Rule 9
    @Rule(StimulusSituation(value='physical object'),
      StimulusResponse(value='hands-on'),
      Feedback(feedback='required'))
    def Rule_9_workshop(self):
        print("Rule 9 fired: Candidate medium = workshop.")
        self.declare(CandidateMedium(
            name="workshop",priority=90,specificity=3,ts=time.time()
        ))

# Rule 10
    @Rule(StimulusSituation(value='symbolic'),
      StimulusResponse(value='analytical'),
      Feedback(feedback='required'))
    def Rule_10_lecture_tutorial_symbolic(self):
        print("Rule 10 fired: Candidate medium = lecture tutorial (symbolic).")
        self.declare(CandidateMedium(
            name="lecture tutorial",priority=80,specificity=3,ts=time.time()
        ))
        
# Rule 11
    @Rule(StimulusSituation(value='visual'),
      StimulusResponse(value='documented'),
      Feedback(feedback='not required'))
    def Rule_11_videocassette(self):
        print("Rule 11 fired: Candidate medium = videocassette.")
        self.declare(CandidateMedium(
          name="videocassette",priority=70,specificity=3,ts=time.time()  
        ))
        
# Rule 12
    @Rule(StimulusSituation(value='visual'),
      StimulusResponse(value='oral'),
      Feedback(feedback='required'))
    def Rule_12_lecture_tutorial_verbal(self):
        print("Rule 12 fired: Candidate medium = lecture tutorial (visual+oral).")
        self.declare(CandidateMedium(
          name="lecture tutorial",priority=85,specificity=3,ts=time.time()  
        ))

# Rule 13
    @Rule(StimulusSituation(value='verbal'),
      StimulusResponse(value='analytical'),
      Feedback(feedback='required'))
    def Rule_13_lecture_tutorial_verbal(self):
        print("Rule 13 fired: Candidate medium = lecture tutorial (verbal+analytical).")
        self.declare(CandidateMedium(
          name="lecture tutorial",priority=75,specificity=3,ts=time.time()  
        ))
        
# Rule 14
    @Rule(StimulusSituation(value='verbal'),
      StimulusResponse(value='oral'),
      Feedback(feedback='required'))
    def Rule_14_role_play(self):
        print("Rule 14 fired: Candidate medium = role-play exercises.")
        self.declare(CandidateMedium(
          name="role-play exercises",priority=78,specificity=3,ts=time.time()  
        ))

# keep the best candidate(A).
    @Rule(AS.c1 << CandidateMedium(),
          AS.c2 << CandidateMedium(),
          TEST(lambda c1,c2:
               (c1["priority"],c1["specificity"],c1["ts"])>=
               (c2["priority"],c2["specificity"],c2["ts"])))
    def resolve_keep_best(self,c1,c2):
        if c1!=c2 : 
            #remove the worse candidate
            self.retract(c2)

# finalize(declare final medium)(B)
    @Rule(AS.c << CandidateMedium())
    def finalize_medium(self,c):
        #Declare final medium once
        if not any(isinstance(f,Medium)for f in self.facts.values()):
            self.declare(Medium(medium=c['name']))
            print(f"\nFinal selected Medium:{c['name']}\n")
advisor = MediaAdvisor()

Initializing the Media Advisor Expert System...



In [5]:
# Test Case 1: Workshop (Physical object, Hands-on, Feedback required)
print("Test Case 1:")
advisor.reset()
advisor.declare(Environment(environment='machines'))
advisor.declare(Job(job='repairing'))
advisor.declare(Feedback(feedback='required'))
advisor.run()

Test Case 1:
Rule 6 fired: job -> stimulus response = hands-on.
Rule 3 fired: Environment -> stimulus situation = physical object.
Rule 9 fired: Candidate medium = workshop.

Final selected Medium:workshop



In [6]:
# Test Case 2: Videocassette (Visual, Documented, No Feedback)
print("Test Case 2:")
advisor.reset()
advisor.declare(Environment(environment='pictures'))
advisor.declare(Job(job='writing'))
advisor.declare(Feedback(feedback='not required'))
advisor.run()

Test Case 2:
Rule 7 fired: job -> stimulus response = documented.
Rule 2 fired: Environment -> stimulus situation = visual.
Rule 11 fired: Candidate medium = videocassette.

Final selected Medium:videocassette



In [7]:
# Test Case 3: Lecture-Tutorial for Symbolic Environment (Symbolic, Analytical, Feedback required)
print("Test Case 3:")
advisor.reset()
advisor.declare(Environment(environment='formulas'))
advisor.declare(Job(job='reasoning'))
advisor.declare(Feedback(feedback='required'))
advisor.run()

Test Case 3:
Rule 8 fired: job -> stimulus response = analytical.
Rule 4 fired: Environment -> stimulus situation = symbolic.


In [8]:
# Test Case 4: Lecture-Tutorial for Verbal Environment (Verbal, Analytical, Feedback required)
print("Test Case 4:")
advisor.reset()
advisor.declare(Environment(environment='manuals'))
advisor.declare(Job(job='investigating'))
advisor.declare(Feedback(feedback='required'))
advisor.run()

Test Case 4:
Rule 8 fired: job -> stimulus response = analytical.
Rule 1 fired: Environment -> stimulus situation = verbal.
Rule 13 fired: Candidate medium = lecture tutorial (verbal+analytical).

Final selected Medium:lecture tutorial



In [9]:
# Test Case 5:Lecture tutorial (Visual, Oral, Feedback required)
print("Test Case 5:")
advisor.reset()
advisor.declare(Environment(environment='diagrams'))
advisor.declare(Job(job='advising'))
advisor.declare(Feedback(feedback='required'))
advisor.run()

Test Case 5:
Rule 5 fired: job -> stimulus response = oral.
Rule 2 fired: Environment -> stimulus situation = visual.
Rule 12 fired: Candidate medium = lecture tutorial (visual+oral).

Final selected Medium:lecture tutorial



In [10]:
# Test Case 6: No Feedback Scenario (Symbolic, Analytical, No Feedback)
print("Test Case 6:")
advisor.reset()
advisor.declare(Environment(environment='computer programs'))
advisor.declare(Job(job='evaluating'))
advisor.declare(Feedback(feedback='not required'))
advisor.run()

Test Case 6:
Rule 8 fired: job -> stimulus response = analytical.
Rule 4 fired: Environment -> stimulus situation = symbolic.


In [11]:
# Test Case 7 (Conflict Scenario): machines + diagrams + repairing + advising + required
print("Test case 7 (conflict):")
advisor.reset()
advisor.declare(Environment(environment="machines"))
advisor.declare(Environment(environment="diagrams"))
advisor.declare(Job(job='repairing'))
advisor.declare(Job(job='advising'))
advisor.declare(Feedback(feedback='required'))
advisor.run()

Test case 7 (conflict):
Rule 5 fired: job -> stimulus response = oral.
Rule 6 fired: job -> stimulus response = hands-on.
Rule 2 fired: Environment -> stimulus situation = visual.
Rule 12 fired: Candidate medium = lecture tutorial (visual+oral).

Final selected Medium:lecture tutorial

Rule 3 fired: Environment -> stimulus situation = physical object.
Rule 9 fired: Candidate medium = workshop.
